In [77]:
import aoc

data = aoc.read("day10.txt", to_grid=True)

# Part 1

In [ ]:
from collections import Counter


def next_step(field, current_direction):

    if field == "F":
        if current_direction == (-1, 0):
            return (0, 1)
        elif current_direction == (0, -1):
            return (1, 0)
    if field == "|":
        if current_direction == (-1, 0):
            return (-1, 0)
        elif current_direction == (1, 0):
            return (1, 0)
    if field == "-":
        if current_direction == (0, 1):

            return (0, 1)
        if current_direction == (0, -1):
            return (0, -1)
    if field == "L":
        if current_direction == (1, 0):
            return (0, 1)
        if current_direction == (0, -1):
            return (-1, 0)
    if field == "J":
        if current_direction == (1, 0):

            return (0, -1)
        if current_direction == (0, 1):

            return (-1, 0)
    if field == "7":

        if current_direction == (-1, 0):
            return (0, -1)
        if current_direction == (0, 1):
            return (1, 0)

    raise ValueError("Non-matching tile")



def find_starting_point(grid):
    n_rows = len(grid)

    n_cols = len(grid[0])
    for row in range(n_rows):
        for col in range(n_cols):

            if grid[row][col] == "S":
                return row, col



def get_starting_direction(starting_row, starting_col, grid):
    size_grid = (len(grid), len(grid[0]))
    acceptable_next_tiles = {
        (0, 1): "7-J",
        (1, 0): "|LJ",
        (0, -1): "-FL",
        (-1, 0): "F|7",
    }
    for try_dir, acceptable in acceptable_next_tiles.items():
        new_row, new_col = starting_row + try_dir[0], starting_col + try_dir[1]
        if (
            not aoc.is_outside_bounds((new_row, new_col), size_grid)
            and grid[new_row][new_col] in acceptable
        ):
            return try_dir
    raise RuntimeError("Could not find acceptable starting direction")


def walk_through_loop(grid):
    row, col = find_starting_point(grid)

    direction = get_starting_direction(row, col, grid)

    field = grid[row][col]
    walked_spots = []
    while True:
        walked_spots.append((row, col))
        row += direction[0]
        col += direction[1]
        field = grid[row][col]
        if field == "S":
            return walked_spots
        direction = next_step(field, direction)



walked_path = walk_through_loop(data)

n_steps_loop = len(walked_path)

farthest_from_s = int((n_steps_loop + 1) / 2)

print(farthest_from_s)

# Part 2

In [ ]:
def find_actual_value_of_S(grid, walked_path):
    cntr = Counter([grid[row][col] for row, col in walked_path])
    if cntr["F"] < cntr["J"]:
        return "F"
    if cntr["J"] < cntr["F"]:
        return "J"
    if cntr["L"] < cntr["7"]:
        return "L"
    if cntr["7"] < cntr["L"]:
        return "7"
    return RuntimeError(f"Could not determine value of `S` for {cntr=}")


def get_inside_direction(current_direction, starts_on_bottom=False):
    if starts_on_bottom:
        seen = {(0, 1): (-1, 0), (1, 0): (0, 1), (-1, 0): (0, -1), (0, -1): (1, 0)}
    else:
        seen = {(0, 1): (1, 0), (1, 0): (0, -1), (-1, 0): (0, 1), (0, -1): (-1, 0)}
    return seen[current_direction]


def get_inside_directions(new_inside_direction, old_inside_direction):
    inside_directions = {new_inside_direction}
    if old_inside_direction != new_inside_direction:
        inside_directions.add(old_inside_direction)
        diag = (
            old_inside_direction[0] + new_inside_direction[0],
            old_inside_direction[1] + new_inside_direction[1],
        )
        inside_directions.add(diag)
    return inside_directions



def find_topleft_visted_f(grid, walked_path):
    n_rows = len(grid)
    n_cols = len(grid[0])
    for row in range(n_rows):
        for col in range(n_cols):
            if grid[row][col] == "F" and (row, col) in walked_path:
                return row, col



def find_inside_fields(grid, walked_path):

    walked_spots_set = set(walked_path)

    true_value_s = find_actual_value_of_S(grid, walked_path)
    row_s, col_s = walked_path[0]
    grid[row_s][col_s] = true_value_s

    # If we start on the topleft F, we know the starting direction and that the enclosed
    # fields are below the current direction
    starting_row, starting_col = find_topleft_visted_f(grid, walked_spots_set)
    direction = (0, 1)

    inside_fields = []
    row, col = starting_row, starting_col

    field = grid[row][col]


    inside_direction = get_inside_direction(direction)

    inside_loc = (row + inside_direction[0], col + inside_direction[1])
    inside_locs = [inside_loc]

    while True:
        for inside_loc in inside_locs:
            if inside_loc not in walked_spots_set:
                if aoc.is_outside_bounds(inside_loc, (len(grid), len(grid[0]))):
                    msg = "This should never happen: an enclosed field outside the grid"
                    raise RuntimeError(msg)

                inside_fields.append(inside_loc)

        row += direction[0]
        col += direction[1]


        field = grid[row][col]

        if (row, col) == (starting_row, starting_col):

            break

        direction = next_step(field, direction)


        new_inside_direction = get_inside_direction(direction)
        inside_directions = get_inside_directions(
            new_inside_direction, inside_direction
        )
        inside_locs = []
        for inside_direction in inside_directions:
            inside_locs.append((row + inside_direction[0], col + inside_direction[1]))
        inside_direction = new_inside_direction

    enclosed_fields = set()
    while inside_fields:
        target = inside_fields.pop()
        if target in enclosed_fields:
            continue
        enclosed_fields.add(target)
        for direction in aoc.DIRECTIONS.values():
            new_loc = target[0] + direction[0], target[1] + direction[1]
            if new_loc not in walked_spots_set:
                inside_fields.append(new_loc)
    return enclosed_fields



enclosed_fields = find_inside_fields(data, walked_path)

print(len(enclosed_fields))

In [ ]:
def part2(filename):
    data = aoc.read(filename, to_grid=True)
    walked_path = walk_through_loop(data)
    enclosed_fields = find_inside_fields(data, walked_path)
    return len(enclosed_fields)


def run_tests():
    aoc.test(part2("day10_example1.txt"), 4)
    aoc.test(part2("day10_example2.txt"), 4)
    aoc.test(part2("day10_example3.txt"), 8)
    aoc.test(part2("day10_example4.txt"), 10)
    print("All tests OK!")


run_tests()